In [ ]:

import numpy as np

import torch

from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from sklearn.model_selection import train_test_split

from src.config import *
from src.data import *
from src.models.efficientnet import EfficientNetB5Custom
from src.utils import *
from src.data import OriginalOAIDataset
from src.train import train, train_model
from src.trainers.classification import Classification


np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)




In [4]:
DATASET_PATH = MENDELEY_OAI_224_SPLIT_PATH
NEW_DATASET_PATH = 'dataset/mendeleyOAI_dataset/brightness_200'
def calculate_brightness(image):
    """
    Calcula el brillo promedio de una imagen.
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    brightness = hsv[:, :, 2].mean()
    return brightness

In [27]:

BRIGHTNESS_THRESHOLD = 200

n_imagenes = 0
n_total_imagenes = 0
imagenes_brillantes = 0
for split in ['train', 'val', 'test']:
    split_path = os.path.join(DATASET_PATH, split)
    for class_name in os.listdir(split_path):
        if not os.path.exists(NEW_DATASET_PATH + '/' + class_name):
            os.makedirs(NEW_DATASET_PATH + '/' + class_name)
        class_path = os.path.join(split_path, class_name)
        total_images = len(os.listdir(class_path))
        n_imagenes = 0
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = cv2.imread(image_path)
            brightness = calculate_brightness(image)
            if brightness < BRIGHTNESS_THRESHOLD:
                cv2.imwrite(NEW_DATASET_PATH + '/' + class_name + '/' + image_name, image)
                n_imagenes += 1
        n_total_imagenes += total_images
        imagenes_brillantes += n_imagenes
        print(f'Clase: {class_name}, Imágenes brillantes: {n_imagenes}/{total_images}, {n_imagenes/total_images*100:.2f}%')
    print(f'Imágenes brillantes: {imagenes_brillantes}/{n_total_imagenes}, {imagenes_brillantes/n_total_imagenes*100:.2f}%')

Clase: 0, Imágenes brillantes: 2201/2286, 96.28%
Clase: 1, Imágenes brillantes: 1007/1046, 96.27%
Clase: 2, Imágenes brillantes: 1426/1516, 94.06%
Clase: 3, Imágenes brillantes: 700/757, 92.47%
Clase: 4, Imágenes brillantes: 159/173, 91.91%
Imágenes brillantes: 5493/5778, 95.07%
Clase: 0, Imágenes brillantes: 306/328, 93.29%
Clase: 1, Imágenes brillantes: 144/153, 94.12%
Clase: 2, Imágenes brillantes: 204/212, 96.23%
Clase: 3, Imágenes brillantes: 100/106, 94.34%
Clase: 4, Imágenes brillantes: 20/27, 74.07%
Imágenes brillantes: 6267/6604, 94.90%
Clase: 0, Imágenes brillantes: 608/639, 95.15%
Clase: 1, Imágenes brillantes: 281/296, 94.93%
Clase: 2, Imágenes brillantes: 426/447, 95.30%
Clase: 3, Imágenes brillantes: 198/223, 88.79%
Clase: 4, Imágenes brillantes: 46/51, 90.20%
Imágenes brillantes: 7826/8260, 94.75%


In [6]:
NEW_DATASET_PATH = 'dataset/mendeleyOAI_dataset/brightness_200_split'
classes = os.listdir(MENDELEY_OAI_BRIGHT_200)

for c in classes:
    class_dir = os.path.join(MENDELEY_OAI_BRIGHT_200, c)

    images = os.listdir(class_dir)
    n_images = len(images)
    n_train = int(n_images * 0.7)
    n_val = int(n_images * 0.1)
    n_test = n_images - n_train - n_val

    print("Generando split para la clase", c)
    print("Train:", n_train)
    print("Val:", n_val)
    print("Test:", n_test)

    train_images, val_test_images = train_test_split(images, test_size=n_val + n_test, random_state=RANDOM_SEED)
    val_images, test_images = train_test_split(val_test_images, test_size=n_test, random_state=RANDOM_SEED)

    if not os.path.exists(os.path.join(NEW_DATASET_PATH, 'train', c)):
        os.makedirs(os.path.join(NEW_DATASET_PATH, 'train', c))
    if not os.path.exists(os.path.join(NEW_DATASET_PATH, 'val', c)):
        os.makedirs(os.path.join(NEW_DATASET_PATH, 'val', c))
    if not os.path.exists(os.path.join(NEW_DATASET_PATH, 'test', c)):
        os.makedirs(os.path.join(NEW_DATASET_PATH, 'test', c))
        

    for img in train_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(NEW_DATASET_PATH, 'train', c, img))
    for img in val_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(NEW_DATASET_PATH, 'val', c, img))
    for img in test_images:
        shutil.copy(os.path.join(class_dir, img), os.path.join(NEW_DATASET_PATH, 'test', c, img))


Generando split para la clase 0
Train: 2180
Val: 311
Test: 624
Generando split para la clase 1
Train: 1002
Val: 143
Test: 287
Generando split para la clase 2
Train: 1439
Val: 205
Test: 412
Generando split para la clase 3
Train: 698
Val: 99
Test: 201
Generando split para la clase 4
Train: 157
Val: 22
Test: 46


In [4]:

ORIGINAL_TRAIN_PATH = os.path.join(MENDELEY_OAI_BRIGHT_200_SPLIT_PATH, 'train')
ORIGINAL_VAL_PATH = os.path.join(MENDELEY_OAI_BRIGHT_200_SPLIT_PATH, 'val')
ORIGINAL_TEST_PATH = os.path.join(MENDELEY_OAI_BRIGHT_200_SPLIT_PATH, 'test')
classes = os.listdir(ORIGINAL_TRAIN_PATH)
print(classes)
data_gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip = True,
    fill_mode='nearest'
)

AUG_MENDELEY_OAI_BRIGHT_200_SPLIT_PATH = 'dataset/mendeleyOAI_dataset/brightness_200_split_aug_3k'


NEW_TRAIN_PATH = os.path.join(AUG_MENDELEY_OAI_BRIGHT_200_SPLIT_PATH, 'train')
NEW_VAL_PATH = os.path.join(AUG_MENDELEY_OAI_BRIGHT_200_SPLIT_PATH, 'val')
NEW_TEST_PATH = os.path.join(AUG_MENDELEY_OAI_BRIGHT_200_SPLIT_PATH, 'test')

# Create new directories
if not os.path.exists(AUG_MENDELEY_OAI_BRIGHT_200_SPLIT_PATH):
    os.makedirs(AUG_MENDELEY_OAI_BRIGHT_200_SPLIT_PATH)
if not os.path.exists(NEW_TRAIN_PATH):
    os.makedirs(NEW_TRAIN_PATH)
if not os.path.exists(NEW_VAL_PATH):
    os.makedirs(NEW_VAL_PATH)
if not os.path.exists(NEW_TEST_PATH):
    os.makedirs(NEW_TEST_PATH)
for split in ['train', 'val', 'test']:
    SPLIT_PATH = os.path.join(AUG_MENDELEY_OAI_BRIGHT_200_SPLIT_PATH, split)
    for class_name in classes:
        CLASS_PATH = os.path.join(SPLIT_PATH, class_name)
        if not os.path.exists(CLASS_PATH):
            os.makedirs(CLASS_PATH)



['0', '1', '2', '3', '4']


In [5]:
AUG_IMAGES = 3000
for class_name in classes:
    CLASS_PATH = os.path.join(NEW_TRAIN_PATH, class_name)
    class_origin = os.path.join(ORIGINAL_TRAIN_PATH, class_name)
    num_augmentations = AUG_IMAGES - len(os.listdir(class_origin))
    

    # Copy original images
    for img_name in os.listdir(class_origin):
        img_path = os.path.join(class_origin, img_name)
        img = cv2.imread(img_path)
        
        # Verificar si la imagen fue leída correctamente
        if img is None:
            print(f"Error al leer la imagen {img_path}. Puede que no sea una imagen válida o esté dañada.")
            continue
        
        
        # Copiar la imagen
        new_img_path = os.path.join(CLASS_PATH, f"{class_name}_{img_name}")
        cv2.imwrite(new_img_path, img)

    print(f"Se han copiado {len(os.listdir(CLASS_PATH))} imágenes de la clase {class_name}")
    print(f'Augmenting {num_augmentations} images for class {class_name}')
    print(f"Generando imágenes aumentadas para la clase {class_name}...")
    
    while(len(os.listdir(CLASS_PATH)) < AUG_IMAGES):
        
        probabilidad = ((AUG_IMAGES - len(os.listdir(CLASS_PATH))) / len(os.listdir(class_origin))) + 0.05
        print(f"Probabilidad de que se genere una imagen aumentada: {probabilidad}")
        for img_name in os.listdir(class_origin):
            if len(os.listdir(CLASS_PATH)) >= AUG_IMAGES:
                break
            if np.random.rand() > probabilidad:
                continue
            
            img_path = os.path.join(class_origin, img_name)
            img = cv2.imread(img_path)
            img_array = img.reshape((1, ) + img.shape)
            for batch in data_gen.flow(img_array, batch_size=1, save_to_dir=CLASS_PATH, save_prefix='aug', save_format='png'):
                break
            
            

    print(f"Se han generado {len(os.listdir(CLASS_PATH))} imágenes aumentadas para la clase {class_name}\n-----------------------------------\n")

Se han copiado 2180 imágenes de la clase 0
Augmenting 820 images for class 0
Generando imágenes aumentadas para la clase 0...
Probabilidad de que se genere una imagen aumentada: 0.4261467889908257
Se han generado 3000 imágenes aumentadas para la clase 0
-----------------------------------

Se han copiado 1002 imágenes de la clase 1
Augmenting 1998 images for class 1
Generando imágenes aumentadas para la clase 1...
Probabilidad de que se genere una imagen aumentada: 2.044011976047904
Probabilidad de que se genere una imagen aumentada: 1.0979041916167664
Probabilidad de que se genere una imagen aumentada: 0.22864271457085827
Se han generado 3000 imágenes aumentadas para la clase 1
-----------------------------------

Se han copiado 1439 imágenes de la clase 2
Augmenting 1561 images for class 2
Generando imágenes aumentadas para la clase 2...
Probabilidad de que se genere una imagen aumentada: 1.1347810979847117
Probabilidad de que se genere una imagen aumentada: 0.20496872828353024
Se ha

In [6]:
for class_name in classes:
    CLASS_PATH = os.path.join(NEW_VAL_PATH, class_name)
    class_origin = os.path.join(ORIGINAL_VAL_PATH, class_name)
    num_augmentations = 1000 - len(os.listdir(class_origin))
    

    # Copy original images
    for img_name in os.listdir(class_origin):
        img_path = os.path.join(class_origin, img_name)
        img = cv2.imread(img_path)
        
        # Verificar si la imagen fue leída correctamente
        if img is None:
            print(f"Error al leer la imagen {img_path}. Puede que no sea una imagen válida o esté dañada.")
            continue
        
        
        # Copiar la imagen
        new_img_path = os.path.join(CLASS_PATH, f"{class_name}_{img_name}")
        cv2.imwrite(new_img_path, img)

    print(f"Se han copiado {len(os.listdir(CLASS_PATH))} imágenes de la clase {class_name}")


for class_name in classes:
    CLASS_PATH = os.path.join(NEW_TEST_PATH, class_name)
    class_origin = os.path.join(ORIGINAL_TEST_PATH, class_name)
    num_augmentations = 1000 - len(os.listdir(class_origin))
    

    # Copy original images
    for img_name in os.listdir(class_origin):
        img_path = os.path.join(class_origin, img_name)
        img = cv2.imread(img_path)
        
        # Verificar si la imagen fue leída correctamente
        if img is None:
            print(f"Error al leer la imagen {img_path}. Puede que no sea una imagen válida o esté dañada.")
            continue
        
        
        # Copiar la imagen
        new_img_path = os.path.join(CLASS_PATH, f"{class_name}_{img_name}")
        cv2.imwrite(new_img_path, img)

    print(f"Se han copiado {len(os.listdir(CLASS_PATH))} imágenes de la clase {class_name}")




Se han copiado 311 imágenes de la clase 0
Se han copiado 143 imágenes de la clase 1
Se han copiado 205 imágenes de la clase 2
Se han copiado 99 imágenes de la clase 3
Se han copiado 22 imágenes de la clase 4
Se han copiado 624 imágenes de la clase 0
Se han copiado 287 imágenes de la clase 1
Se han copiado 412 imágenes de la clase 2
Se han copiado 201 imágenes de la clase 3
Se han copiado 46 imágenes de la clase 4


In [7]:
img = explorar_split_data(AUG_MENDELEY_OAI_BRIGHT_200_SPLIT_PATH)

╒═════════╤══════╤══════╤══════╤══════╤══════╕
│ Clase   │    0 │    1 │    2 │    3 │    4 │
╞═════════╪══════╪══════╪══════╪══════╪══════╡
│ train   │ 3000 │ 3000 │ 3000 │ 3000 │ 3000 │
├─────────┼──────┼──────┼──────┼──────┼──────┤
│ val     │  311 │  143 │  205 │   99 │   22 │
├─────────┼──────┼──────┼──────┼──────┼──────┤
│ test    │  624 │  287 │  412 │  201 │   46 │
╘═════════╧══════╧══════╧══════╧══════╧══════╛
